In [1]:
from transformers import pipeline
import pickle
import json
import pandas as pd
import ast


import tensorflow as tf
import tensorflow_hub as hub
#import tensorflow_text as text

/Users/emilychen/opt/anaconda3/envs/fbads/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import requests
from bs4 import BeautifulSoup
import re

## Election Ads

In [3]:
#new_df = pd.read_csv('emily.csv.crdownload')
new_df = pd.read_csv('election_sentiment.csv')
new_df.head()

,Unnamed: 0,Unnamed: 0.1,ad_creative_bodies,demographic_distribution,delivery_by_region,estimated_audience_size,spend,impressions,cumulative_impressions_by_region,cum_spend,sentiment
0,0,0,,"[[{'percentage': '0.000876', 'age': '45-54', '...","[[{'percentage': '1', 'region': 'New York'}], ...","[{'lower_bound': '10001', 'upper_bound': '5000...","[{'lower_bound': '300', 'upper_bound': '399'},...","[{'lower_bound': '8000', 'upper_bound': '8999'...","{'New York': 28498.5, 'Georgia': 7999.0, 'Conn...",1347.5,POSITIVE
1,1,1,!!!The government cannot fight CORRUPTION - th...,"[[{'percentage': '0.000933', 'age': '18-24', '...","[[{'percentage': '0.00185', 'region': 'Rhode I...",[{'lower_bound': '1000001'}],"[{'lower_bound': '0', 'upper_bound': '99'}]","[{'lower_bound': '1000', 'upper_bound': '1999'}]","{'Rhode Island': 1499.5, 'New York': 1499.5, '...",49.5,NEGATIVE
2,2,2,!!🚨Endorsement Alert🚨!! \n\nThank you Byron Po...,"[[{'percentage': '0.004039', 'age': '65+', 'ge...","[[{'percentage': '0.002281', 'region': 'Unknow...","[{'lower_bound': '100001', 'upper_bound': '500...","[{'lower_bound': '100', 'upper_bound': '199'}]","[{'lower_bound': '2000', 'upper_bound': '2999'}]","{'Unknown': 2499.5, 'New Mexico': 2499.5}",149.5,POSITIVE
3,3,3,""" An astute CPA told me 45 years ago, Math nev...","[[{'percentage': '0.00112', 'age': '18-24', 'g...","[[{'percentage': '1', 'region': 'Ohio'}], [{'p...","[{'lower_bound': '100001', 'upper_bound': '500...","[{'lower_bound': '0', 'upper_bound': '99'}, {'...","[{'lower_bound': '2000', 'upper_bound': '2999'...",{'Ohio': 5999.0},99.0,POSITIVE
4,4,4,"""....Framingham should not go back to the ways...","[[{'percentage': '0.004255', 'age': '45-54', '...","[[{'percentage': '1', 'region': 'Massachusetts...","[{'lower_bound': '50001', 'upper_bound': '1000...","[{'lower_bound': '0', 'upper_bound': '99'}, {'...","[{'lower_bound': '0', 'upper_bound': '999'}, {...","{'Massachusetts': 999.0, 'Connecticut': 499.5,...",99.0,POSITIVE


## Election Ads - Sentiment Analysis

In [4]:
def find_label(x):
    for class_dict in classifier(x[:512]):
        return(class_dict['label'])

In [5]:
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
2022-05-06 16:01:26.581177: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-06 16:01:26.636334: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fbada53eb90 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-05-06 16:01:26.636360: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-05-06 16:01:26.676167: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
A

In [6]:
#new_df['sentiment'] = new_df.get('ad_creative_bodies').apply(find_label)
new_df['cumulative_impressions_by_region'] = new_df['cumulative_impressions_by_region'].apply(ast.literal_eval)

In [7]:
new_df.head()

,Unnamed: 0,Unnamed: 0.1,ad_creative_bodies,demographic_distribution,delivery_by_region,estimated_audience_size,spend,impressions,cumulative_impressions_by_region,cum_spend,sentiment
0,0,0,,"[[{'percentage': '0.000876', 'age': '45-54', '...","[[{'percentage': '1', 'region': 'New York'}], ...","[{'lower_bound': '10001', 'upper_bound': '5000...","[{'lower_bound': '300', 'upper_bound': '399'},...","[{'lower_bound': '8000', 'upper_bound': '8999'...","{'New York': 28498.5, 'Georgia': 7999.0, 'Conn...",1347.5,POSITIVE
1,1,1,!!!The government cannot fight CORRUPTION - th...,"[[{'percentage': '0.000933', 'age': '18-24', '...","[[{'percentage': '0.00185', 'region': 'Rhode I...",[{'lower_bound': '1000001'}],"[{'lower_bound': '0', 'upper_bound': '99'}]","[{'lower_bound': '1000', 'upper_bound': '1999'}]","{'Rhode Island': 1499.5, 'New York': 1499.5, '...",49.5,NEGATIVE
2,2,2,!!🚨Endorsement Alert🚨!! \n\nThank you Byron Po...,"[[{'percentage': '0.004039', 'age': '65+', 'ge...","[[{'percentage': '0.002281', 'region': 'Unknow...","[{'lower_bound': '100001', 'upper_bound': '500...","[{'lower_bound': '100', 'upper_bound': '199'}]","[{'lower_bound': '2000', 'upper_bound': '2999'}]","{'Unknown': 2499.5, 'New Mexico': 2499.5}",149.5,POSITIVE
3,3,3,""" An astute CPA told me 45 years ago, Math nev...","[[{'percentage': '0.00112', 'age': '18-24', 'g...","[[{'percentage': '1', 'region': 'Ohio'}], [{'p...","[{'lower_bound': '100001', 'upper_bound': '500...","[{'lower_bound': '0', 'upper_bound': '99'}, {'...","[{'lower_bound': '2000', 'upper_bound': '2999'...",{'Ohio': 5999.0},99.0,POSITIVE
4,4,4,"""....Framingham should not go back to the ways...","[[{'percentage': '0.004255', 'age': '45-54', '...","[[{'percentage': '1', 'region': 'Massachusetts...","[{'lower_bound': '50001', 'upper_bound': '1000...","[{'lower_bound': '0', 'upper_bound': '99'}, {'...","[{'lower_bound': '0', 'upper_bound': '999'}, {...","{'Massachusetts': 999.0, 'Connecticut': 499.5,...",99.0,POSITIVE


In [8]:
#new_df.to_csv('election_sentiment.csv')

# Visualizing Positive Election Ads

In [9]:
us_states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Washington, District of Columbia', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

In [10]:
df_list = []
for index, row in new_df.iterrows():
    curr_dict = row['cumulative_impressions_by_region']
    d = pd.DataFrame({'region': curr_dict.keys(), 'impressions': curr_dict.values()})
    d['sentiment'] = row['sentiment']
    df_list.append(d)
print(len(df_list))

11737


In [11]:
positive_df = pd.concat(df_list, ignore_index = True)
positive_df = positive_df[positive_df['sentiment'] == 'POSITIVE']
positive_df.head()

,region,impressions,sentiment
0,New York,28498.5,POSITIVE
1,Georgia,7999.0,POSITIVE
2,Connecticut,74999.5,POSITIVE
3,Massachusetts,74999.5,POSITIVE
4,Alaska,2499.5,POSITIVE


In [12]:
positive_df.shape

(83603, 3)

In [13]:
positive_df = positive_df.groupby('region').sum().reset_index()
positive_df = positive_df[positive_df.get('region').isin(us_states) == True]
positive_df.head()

,region,impressions
21,Alabama,72297671.5
22,Alaska,67060064.5
42,Arizona,75921804.0
43,Arkansas,70117770.5
129,California,83010087.0


In [14]:
census = pd.read_csv('censusdata.csv')
census = census[census['NAME'].isin(us_states)]
census.head()

,Unnamed: 0,NAME,POPESTIMATE2020,POPESTIMATE2021
5,5,Alabama,5024803,5039877
6,6,Alaska,732441,732673
7,7,Arizona,7177986,7276316
8,8,Arkansas,3012232,3025891
9,9,California,39499738,39237836


In [15]:
# standardizing
positive_df = positive_df.merge(census, left_on = 'region', right_on = 'NAME', how = 'outer')
positive_df.head()

,region,impressions,Unnamed: 0,NAME,POPESTIMATE2020,POPESTIMATE2021
0,Alabama,72297671.5,5.0,Alabama,5024803.0,5039877.0
1,Alaska,67060064.5,6.0,Alaska,732441.0,732673.0
2,Arizona,75921804.0,7.0,Arizona,7177986.0,7276316.0
3,Arkansas,70117770.5,8.0,Arkansas,3012232.0,3025891.0
4,California,83010087.0,9.0,California,39499738.0,39237836.0


In [16]:
positive_df.shape

(51, 6)

In [17]:
positive_df.get('region').iloc[47] = 'District of Columbia'
positive_df.sort_values(by = 'region', inplace = True)

/Users/emilychen/opt/anaconda3/envs/fbads/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [18]:
import altair as alt
import geopandas as gpd
import json
from vega_datasets import data
pop = data.population_engineers_hurricanes()
positive_df['id'] = list(pop.get('id'))[:-1]
positive_df.head()

,region,impressions,Unnamed: 0,NAME,POPESTIMATE2020,POPESTIMATE2021,id
0,Alabama,72297671.5,5.0,Alabama,5024803.0,5039877.0,1
1,Alaska,67060064.5,6.0,Alaska,732441.0,732673.0,2
2,Arizona,75921804.0,7.0,Arizona,7177986.0,7276316.0,4
3,Arkansas,70117770.5,8.0,Arkansas,3012232.0,3025891.0,5
4,California,83010087.0,9.0,California,39499738.0,39237836.0,6


In [19]:
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [20]:
states = alt.topo_feature(data.us_10m.url, feature='states')

alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=500,
    height=300
)

alt.Chart(...)

In [21]:
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, 'states')

variable_list = ['impressions']

# Creating configs for color,selection,hovering
hover = alt.selection(type='single', on='mouseover', nearest=True,
                      fields=['x', 'y'])

plot = alt.Chart(states).mark_geoshape().encode(
    color='impressions:Q'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(positive_df, 'id', list(positive_df.columns))
).properties(
    width=500,
    height=300,
    title = 'Cumulative Impressions: Positive Election Ads'
).project(
    type='albersUsa'
)

outline = alt.Chart(states).mark_geoshape(stroke='black', fillOpacity=0).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

alt.layer(plot,outline)

alt.LayerChart(...)

# Visualize Negative Election Ads

In [22]:
negative_df = pd.concat(df_list, ignore_index = True)
negative_df = negative_df[negative_df['sentiment'] == 'NEGATIVE']
negative_df = negative_df.groupby('region').sum().reset_index()
negative_df = negative_df[negative_df.get('region').isin(us_states) == True]
negative_df.head()

,region,impressions
11,Alabama,79695297.5
13,Alaska,78239485.5
20,Arizona,81300017.5
21,Arkansas,81225834.0
57,California,88805393.5


In [23]:
negative_df.shape

(51, 2)

In [24]:
negative_df.get('region').iloc[47] = 'District of Columbia'
negative_df.sort_values(by = 'region', inplace = True)

In [25]:
pop = data.population_engineers_hurricanes()
negative_df['id'] = list(pop.get('id'))[:-1]
negative_df.head()

,region,impressions,id
11,Alabama,79695297.5,1
13,Alaska,78239485.5,2
20,Arizona,81300017.5,4
21,Arkansas,81225834.0,5
57,California,88805393.5,6


In [26]:
alt.renderers.enable('default')

from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, 'states')

variable_list = ['impressions']

# Creating configs for color,selection,hovering
hover = alt.selection(type='single', on='mouseover', nearest=True,
                      fields=['x', 'y'])

plot = alt.Chart(states).mark_geoshape().encode(
    color='impressions:Q'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(negative_df, 'id', list(negative_df.columns))
).properties(
    width=500,
    height=300,
    title = 'Cumulative Impressions: Negative Election Ads'
).project(
    type='albersUsa'
)

outline = alt.Chart(states).mark_geoshape(stroke='black', fillOpacity=0).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

alt.layer(plot,outline)

alt.LayerChart(...)

# Hypothesis Test: Election Data

In [27]:
df_list = []
for index, row in new_df.iterrows():
    curr_dict = row['cumulative_impressions_by_region']
    d = pd.DataFrame({'region': curr_dict.keys(), 'impressions': curr_dict.values()})
    d['sentiment'] = row['sentiment']
        
    df_list.append(d)
print(len(df_list))

11737


In [28]:
z_test_table = pd.concat(df_list, ignore_index = True)

In [29]:
z_test_table = z_test_table[z_test_table['region'].isin(us_states)]
z_test_table.head()

,region,impressions,sentiment
0,New York,28498.5,POSITIVE
1,Georgia,7999.0,POSITIVE
2,Connecticut,74999.5,POSITIVE
3,Massachusetts,74999.5,POSITIVE
4,Alaska,2499.5,POSITIVE


In [30]:
z_test_table.shape

(139880, 3)

In [31]:
# based on 2020 election
right_wing = ['Alabama', 'Alaska', 'Arkansas', 'Florida', 'Idaho', 'Indiana', 
              'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Mississippi', 'Missouri', 
              'Montana', 'Nebraska', 'North Carolina', 'North Dakota', 'Ohio', 
              'Oklahoma', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 
              'Utah', 'West Virginia', 'Wyoming']
left_wing = ['Arizona', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Georgia', 
             'Hawaii', 'Illinois', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 
             'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'Oregon', 
             'Pennsylvania', 'Rhode Island', 'Vermont', 'Virginia', 'Washington', 'Wisconsin']

In [32]:
z_test_table[z_test_table['sentiment'] == 'NEGATIVE'].head()

,region,impressions,sentiment
47,Rhode Island,1499.5,NEGATIVE
48,New York,1499.5,NEGATIVE
49,Kansas,1499.5,NEGATIVE
50,Louisiana,1499.5,NEGATIVE
51,Maine,1499.5,NEGATIVE


In [33]:
# total number of ads in right wing
print(z_test_table[z_test_table['region'].isin(right_wing)].shape)

(65623, 3)


In [34]:
# total number of ads in left wing
print(z_test_table[z_test_table['region'].isin(left_wing)].shape)

(72032, 3)


In [35]:
# number of negative ads in right wing
z_test_table[(z_test_table['region'].isin(right_wing)) 
             & (z_test_table['sentiment'] == 'NEGATIVE')].shape

(27323, 3)

In [36]:
# number of negative ads in left wing
z_test_table[z_test_table['region'].isin(left_wing) 
             & (z_test_table['sentiment'] == 'NEGATIVE')].shape

(29535, 3)

In [37]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

In [38]:
print(27323/65623)
print(29535/72032)

0.4163631653536108
0.4100260995113283


In [39]:
count = [27323, 29535]# negative ads for both wings 
nobs = [65623, 72032] # total ads in right wing, total ads in left wing 
#value = 0.05
stat, pval = proportions_ztest(count, nobs)
print(pval)

0.01708085484697255


# Hypothesis Test - Vaccines

In [42]:
#vaccines = pd.read_csv('emily_vaccine.csv')
vaccines = pd.read_csv('vaccine_sentiment.csv')
vaccines.head()

,Unnamed: 0,Unnamed: 0.1,ad_creative_bodies,demographic_distribution,delivery_by_region,estimated_audience_size,spend,impressions,cumulative_impressions_by_region,cum_spend,sentiment
0,0,0,,"[[{'percentage': '0.001006', 'age': '13-17', '...","[[{'percentage': '0.001516', 'region': 'Rhode ...","[{'lower_bound': '5001', 'upper_bound': '10000...","[{'lower_bound': '1000', 'upper_bound': '1499'...","[{'lower_bound': '25000', 'upper_bound': '2999...","{'Rhode Island': 2231992.5, 'Oregon': 2231992....",48641.0,POSITIVE
1,1,1,!!!!!!!!!!!!!!!!!!!!,"[[{'percentage': '4.9E-5', 'age': '18-24', 'ge...","[[{'percentage': '0.034184', 'region': 'Alabam...",[{'lower_bound': '1000001'}],"[{'lower_bound': '0', 'upper_bound': '99'}]","[{'lower_bound': '25000', 'upper_bound': '2999...","{'Alabama': 27499.5, 'Rhode Island': 27499.5, ...",49.5,POSITIVE
2,2,2,"""'Safe, potent, & effective' - those three wor...","[[{'percentage': '5.4E-5', 'age': '13-17', 'ge...","[[{'percentage': '0.000163', 'region': 'Unknow...",[{'lower_bound': '1000001'}],"[{'lower_bound': '300', 'upper_bound': '399'}]","[{'lower_bound': '15000', 'upper_bound': '1999...","{'Unknown': 17499.5, 'Pennsylvania': 17499.5, ...",349.5,POSITIVE
3,3,3,"""1 in 500 Americans have died from COVID-19. O...","[[{'percentage': '0.000943', 'age': '45-54', '...","[[{'percentage': '1', 'region': 'Texas'}], [{'...","[{'lower_bound': '10001', 'upper_bound': '5000...","[{'lower_bound': '0', 'upper_bound': '99'}, {'...","[{'lower_bound': '1000', 'upper_bound': '1999'...",{'Texas': 17496.5},346.5,NEGATIVE
4,4,4,"""A patent seizure is largely symbolic rather t...","[[{'percentage': '0.000456', 'age': '18-24', '...","[[{'percentage': '0.01887', 'region': 'Alabama...",[{'lower_bound': '1000001'}],"[{'lower_bound': '200', 'upper_bound': '299'}]","[{'lower_bound': '20000', 'upper_bound': '2499...","{'Alabama': 22499.5, 'Rhode Island': 22499.5, ...",249.5,NEGATIVE


In [43]:
#vaccines['sentiment'] = vaccines.get('ad_creative_bodies').apply(find_label)
vaccines['cumulative_impressions_by_region'] = vaccines['cumulative_impressions_by_region'].apply(ast.literal_eval)

In [44]:
#vaccines.to_csv('vaccine_sentiment.csv')

In [45]:
vacc_df_list = []
for index, row in vaccines.iterrows():
    curr_dict = row['cumulative_impressions_by_region']
    d = pd.DataFrame({'region': curr_dict.keys(), 'impressions': curr_dict.values()})
    d['sentiment'] = row['sentiment']
        
    vacc_df_list.append(d)
print(len(vacc_df_list))

7248


In [46]:
vacc_z_test_table = pd.concat(vacc_df_list, ignore_index = True)
vacc_z_test_table = vacc_z_test_table[vacc_z_test_table['region'].isin(us_states)]
vacc_z_test_table.head()

,region,impressions,sentiment
0,Rhode Island,2231992.5,POSITIVE
1,Oregon,2231992.5,POSITIVE
2,Nebraska,2231992.5,POSITIVE
3,Nevada,2231992.5,POSITIVE
4,New Hampshire,2231992.5,POSITIVE


In [47]:
# total number of ads in right wing
print(vacc_z_test_table[vacc_z_test_table['region'].isin(right_wing)].shape)

(63620, 3)


In [48]:
# total number of ads in left wing
print(vacc_z_test_table[vacc_z_test_table['region'].isin(left_wing)].shape)

(65936, 3)


In [49]:
# number of negative ads in right wing
vacc_z_test_table[(vacc_z_test_table['region'].isin(right_wing)) 
             & (vacc_z_test_table['sentiment'] == 'NEGATIVE')].shape

(37398, 3)

In [50]:
# number of negative ads in left wing
vacc_z_test_table[vacc_z_test_table['region'].isin(left_wing) 
             & (vacc_z_test_table['sentiment'] == 'NEGATIVE')].shape

(38660, 3)

In [55]:
65936+63620

129556

In [51]:
print(38660/65936)
print(37398/63620)

0.58632613443339
0.5878340144608614


In [52]:
count = [38660, 37398]# negative ads for both wings 
nobs = [65936, 63620] # total ads in left wing, total ads in right wing 
#value = 0.05
stat, pval = proportions_ztest(count, nobs)
print(pval)

0.5815807383468412
